In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
df=pd.read_csv('PenDigits_withoutdupl_norm_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,outlier
0,48.0,100.0,20.0,80.0,0.0,51.0,55.0,43.0,100.0,56.0,90.0,62.0,86.0,30.0,73.0,0.0,1
1,38.0,100.0,4.0,85.0,0.0,53.0,58.0,46.0,100.0,52.0,88.0,64.0,78.0,32.0,73.0,0.0,1
2,53.0,100.0,25.0,70.0,0.0,39.0,50.0,34.0,100.0,42.0,85.0,64.0,77.0,32.0,80.0,0.0,1
3,12.0,100.0,0.0,71.0,25.0,43.0,86.0,44.0,100.0,75.0,90.0,52.0,90.0,20.0,90.0,0.0,1
4,26.0,100.0,0.0,83.0,23.0,51.0,83.0,42.0,100.0,55.0,79.0,84.0,56.0,42.0,37.0,0.0,1


In [3]:
# df to values
df = df.values

In [4]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/PenDigits.json")  
gc = GCForest(config)

In [16]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,16]
X = df[:,0:16]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
# count class
len(np.unique(y_train))

2

In [7]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 09:42:19,011][cascade_classifier.fit_transform] X_groups_train.shape=[(6907, 16)],y_train.shape=(6907,),X_groups_test.shape=[(2961, 16)],y_test.shape=(2961,)
[ 2018-04-25 09:42:19,013][cascade_classifier.fit_transform] group_dims=[16]
[ 2018-04-25 09:42:19,015][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:42:19,016][cascade_classifier.fit_transform] group_ends=[16]
[ 2018-04-25 09:42:19,017][cascade_classifier.fit_transform] X_train.shape=(6907, 16),X_test.shape=(2961, 16)
[ 2018-04-25 09:42:19,019][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6907, 16), X_cur_test.shape=(2961, 16)
[ 2018-04-25 09:42:19,682][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.86%
[ 2018-04-25 09:42:20,499][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 09:42:21,281][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

In [8]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 09:42:27,134][cascade_classifier.transform] X_groups_test.shape=[(2961, 16)]
[ 2018-04-25 09:42:27,137][cascade_classifier.transform] group_dims=[16]
[ 2018-04-25 09:42:27,138][cascade_classifier.transform] X_test.shape=(2961, 16)
[ 2018-04-25 09:42:27,141][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2961, 16)


Test Accuracy of ExtratTrees = 99.966228 %


In [9]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 09:42:28,685][cascade_classifier.transform] X_groups_test.shape=[(2961, 16)]
[ 2018-04-25 09:42:28,687][cascade_classifier.transform] group_dims=[16]
[ 2018-04-25 09:42:28,690][cascade_classifier.transform] X_test.shape=(2961, 16)
[ 2018-04-25 09:42:28,692][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2961, 16)


In [10]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [11]:
X_test_new.shape

(2961, 18)

In [12]:
X_train_new.shape

(6907, 18)

In [13]:
# Cascade ExtraTrees
config = load_json("./examples/PenDigitsgc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 09:42:30,263][cascade_classifier.fit_transform] X_groups_train.shape=[(6907, 18)],y_train.shape=(6907,),X_groups_test.shape=[(2961, 18)],y_test.shape=(2961,)
[ 2018-04-25 09:42:30,265][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-25 09:42:30,266][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:42:30,268][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-25 09:42:30,270][cascade_classifier.fit_transform] X_train.shape=(6907, 18),X_test.shape=(2961, 18)
[ 2018-04-25 09:42:30,272][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6907, 18), X_cur_test.shape=(2961, 18)
[ 2018-04-25 09:42:30,972][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.86%
[ 2018-04-25 09:42:31,766][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 09:42:32,537][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

[ 2018-04-25 09:43:03,972][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.71%
[ 2018-04-25 09:43:04,754][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.86%
[ 2018-04-25 09:43:05,567][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=99.86%
[ 2018-04-25 09:43:06,332][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 09:43:07,333][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.86%
[ 2018-04-25 09:43:08,152][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.86%
[ 2018-04-25 09:43:08,923][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 09:43:09,724][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.0

In [14]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [15]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/PenDigitsgc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 09:43:10,772][cascade_classifier.fit_transform] X_groups_train.shape=[(6907, 18)],y_train.shape=(6907,),X_groups_test.shape=[(2961, 18)],y_test.shape=(2961,)
[ 2018-04-25 09:43:10,775][cascade_classifier.fit_transform] group_dims=[18]
[ 2018-04-25 09:43:10,776][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 09:43:10,778][cascade_classifier.fit_transform] group_ends=[18]
[ 2018-04-25 09:43:10,779][cascade_classifier.fit_transform] X_train.shape=(6907, 18),X_test.shape=(2961, 18)
[ 2018-04-25 09:43:10,782][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6907, 18), X_cur_test.shape=(2961, 18)
[ 2018-04-25 09:43:11,388][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.71%
[ 2018-04-25 09:43:12,191][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 09:43:12,971][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 

[ 2018-04-25 09:43:45,283][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=99.71%
[ 2018-04-25 09:43:46,098][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=99.86%
[ 2018-04-25 09:43:46,904][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=100.00%
[ 2018-04-25 09:43:47,713][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 09:43:48,556][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=99.86%
[ 2018-04-25 09:43:49,335][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.71%
[ 2018-04-25 09:43:50,147][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 09:43:50,966][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.

('Test Accuracy of Extratrees = 100.000000 %', 0, 'fois')


[ 2018-04-25 09:43:55,847][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.71%
[ 2018-04-25 09:43:56,624][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 09:43:57,447][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.71%
[ 2018-04-25 09:43:58,345][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.86%
[ 2018-04-25 09:43:59,114][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=100.00%
[ 2018-04-25 09:43:59,945][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.71%
[ 2018-04-25 09:44:00,754][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 09:44:01,557][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.8

[ 2018-04-25 09:44:33,379][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.71%
[ 2018-04-25 09:44:34,201][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 09:44:35,046][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00%
[ 2018-04-25 09:44:35,937][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=100.00%
[ 2018-04-25 09:44:36,077][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=99.88%
[ 2018-04-25 09:44:36,079][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=100.00%
[ 2018-04-25 09:44:36,081][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=99.88%
[ 2018-04-25 09:44:36,082][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=100.00%
[ 2018-04-25 09:44:36,

...

[ 2018-04-25 09:58:07,142][cascade_classifier.fit_transform] X_train.shape=(6907, 18),X_test.shape=(2961, 18)
[ 2018-04-25 09:58:07,144][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(6907, 18), X_cur_test.shape=(2961, 18)


('Test Accuracy of Extratrees = 99.966228 %', 18, 'fois')


[ 2018-04-25 09:58:07,758][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.71%
[ 2018-04-25 09:58:08,594][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=100.00%
[ 2018-04-25 09:58:09,368][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.71%
[ 2018-04-25 09:58:10,123][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.86%
[ 2018-04-25 09:58:10,889][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.86%
[ 2018-04-25 09:58:11,704][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.71%
[ 2018-04-25 09:58:12,493][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=100.00%
[ 2018-04-25 09:58:13,255][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=100.0

[ 2018-04-25 09:58:45,096][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=99.71%
[ 2018-04-25 09:58:45,859][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=100.00%
[ 2018-04-25 09:58:46,646][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=100.00%
[ 2018-04-25 09:58:47,484][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=99.86%
[ 2018-04-25 09:58:47,635][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=99.87%
[ 2018-04-25 09:58:47,637][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=100.00%
[ 2018-04-25 09:58:47,639][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=99.87%
[ 2018-04-25 09:58:47,641][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=100.00%
[ 2018-04-25 09:58:47,6

('Test Accuracy of Extratrees = 100.000000 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 100.000000 %', 0, 'fois')